In [1]:
import nltk
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader
from nltk.book import FreqDist
import os
import re
import community.community_louvain as community_louvain
import math
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
#from PIL import Image

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [2]:
df_aesthetics = pd.read_csv('aesthetics.csv')

In [3]:
file_list=[]

for aest in list(df_aesthetics['0']):
    aest_file = aest.replace('/','_')
    file_list+=[aest_file+".txt"]

In [4]:
#file_list

In [5]:
corpus_root = os.getcwd()+'/clean'

wordlists = PlaintextCorpusReader(corpus_root, file_list)

wholeText = nltk.Text(wordlists.words())

OSError: No such file or directory: 'C:\\Users\\Gabo\\Desktop\\Social Graphs\\Final project\\SG Final\\clean\\Arts and Crafts.txt'

In [ ]:
len(wholeText)

In [ ]:
len(set(wholeText))

In [ ]:
len(set(wholeText)) / len(wholeText)

In [ ]:
fdist1 = FreqDist(wholeText)
fdist1

In [ ]:
plt.figure(figsize=(20, 15))
fdist1.plot(75, cumulative=True)

In [ ]:
fdist1.most_common(5)

In [ ]:
corpus_root = os.getcwd()+'/cleaner'

wordlists_nostop = PlaintextCorpusReader(corpus_root, file_list)

nostopText = nltk.Text(wordlists_nostop.words())

In [ ]:
len(nostopText)

In [ ]:
len(set(nostopText))

In [ ]:
len(set(nostopText)) / len(nostopText)

In [ ]:
fdist2 = FreqDist(nostopText)
fdist2

In [ ]:
plt.figure(figsize=(20, 15))
fdist2.plot(75, cumulative=True)

In [ ]:
fdist2.most_common(50)

In [ ]:
disp = ['aesthetic', 'fashion', 'music', 'style','art', 'people', 'colors']

In [ ]:
wholeText.dispersion_plot(disp)

In [ ]:
disp2 = ['architecture', 'music', 'fashion', 'art', 'culture', 'nature']

In [ ]:
wholeText.dispersion_plot(disp2)

In [ ]:
wholeText.collocations()

In [ ]:
alph_list

In [ ]:
DG = nx.DiGraph()

for i in list(df_aesthetics['0']):
    if "|" in i:
        spl=re.split('(?:\|)', i)
        alph_list[alph_list.index(i)]=spl[1]
        DG.add_node(spl[1])    
    else:
        DG.add_node(i)

In [ ]:
len(DG.nodes())

In [ ]:
egdes={}
for char in DG:
    char_file = char.replace('/','_')
    with open('aesthetics/'+f'{char_file}.txt', 'r') as f:
        a = f.read().replace('\n', '')
        pol = list(set(re.findall(r'\[\[([\w\s]*).*?\]\]', a)))
    table = str(re.findall(r'(?<=\{\{).*?(?=\}\})', a))

    
    connections = [x for x in list(set(re.findall(r'(?<=\[\[).*?(?=\]\])', table))) if x in DG]
    
    for c in connections:
        DG.add_edge(char, c )

In [ ]:
len(DG.edges())

In [ ]:
UG = DG.to_undirected()


In [ ]:
partition = community_louvain.best_partition(UG)
len(set(partition.values()))

In [ ]:
community_louvain.modularity(partition, UG)

In [ ]:
plt.hist(partition.values(), len(set(partition.values())))
plt.show()

In [ ]:
sums = {}
coms = []
for x in set(partition.values()):
    sums[x] = sum(value == x for value in partition.values())
    coms.append([k for k,v in partition.items() if v == x])

In [ ]:
#sums
#coms[31]

In [ ]:
big_coms = [x[0] for x in sorted(sums.items(), key=lambda x: x[1], reverse=True)[:5]]

In [ ]:
big_coms.sort()
big_coms

In [ ]:
nx.set_node_attributes(UG, partition, "partition")

In [ ]:
#Sorting the characters by edges and communities
ch_1=[]
for n in UG.nodes():
    ch_1.append((n,UG.degree(n),UG.nodes()[n]['partition']))

ch_1.sort(key=lambda x:x[1],reverse=True)
ch_1.sort(key=lambda x:x[2])

In [ ]:
ch_big = [x for x in ch_1 if x[2] in big_coms]

In [ ]:
ch_big

In [ ]:
#Name each community by its three most connected characters.
count=0
com_name={}
for i in range(len(ch_big)):
    if ch_big[i][2]==big_coms[count]:
        com_name[big_coms[count]]=str(ch_big[i][0])+"-"+str(ch_big[i+1][0])+"-"+str(ch_big[i+2][0])
        count+=1
    

In [ ]:
com_name

In [ ]:
#ch_1

In [ ]:
community_corpora={}

for com in [x[0] for x in sorted(sums.items(), key=lambda x: x[1], reverse=True)]:
    char = coms[com]
    
    char = [x.replace('/','_') for x in char]
    
    clist = [i+".txt" for i in char]
    
    wordlists_ = PlaintextCorpusReader(corpus_root, clist)
    community_corpora[com]= nltk.Text(wordlists_.words())

In [ ]:
#community_corpora

In [ ]:
#TF values ft/len(text) and idf values log((N/nt)
comunity_tf={}
for x in big_coms:
    l=[]
    l = sorted(dict(FreqDist(community_corpora[x])).items(), key=lambda x: x[1], reverse=True)
    d={}
    for y in l:
        d[y[0]] = y[1]/len(community_corpora[x])
    comunity_tf[x] = d

In [ ]:
for r in comunity_tf:
    print(sorted(comunity_tf[r].items(), key = lambda x: x[1], reverse = True)[:5])

In [ ]:
N_com = len(community_corpora)

comunity_tfidf = comunity_tf.copy()
idf_com = {}

for word in set(nostopText):
    nt = 0
    for comu in community_corpora:
        if word in set(community_corpora[comu]):
            nt+=1
    idf_com[word] = math.log((N_com/(nt+1)),10)
    
for comu in comunity_tfidf:
    for word in comunity_tfidf[comu]:
        comunity_tfidf[comu][word] = int(comunity_tfidf[comu][word]*idf_com[word]*10000) # multiplying by 10000 and changing into integer to prepare the scores for wordclouds

In [ ]:
for r in comunity_tfidf:
    print(sorted(comunity_tfidf[r].items(), key = lambda x: x[1], reverse = True)[:5])

In [ ]:
wc_str = {}

for comu in big_coms:
    wc_str[comu] = ''

    for x in comunity_tfidf[comu]:
        wc_str[comu] += (x+' ')*comunity_tfidf[comu][x]

In [ ]:
wc_str

In [ ]:
plt.figure()
stopwords = set(STOPWORDS)
plt.figure(figsize=[17, 30])

for t in range(len(big_coms)):
    plt.subplot(5, 1, t+1).set_title(str(big_coms[t])+" wordcloud")
    
#     mask = np.array(Image.open(races_wc[t]+".png"))
#     transformed_mask = np.ndarray((mask.shape[0],mask.shape[1],mask.shape[2]), np.int32)
#     for i in range(len(mask)):
#         for j in range(len(mask[i])):
#             transformed_mask[i][j] = list(map(transform_format, mask[i][j]))
            
#     image_colors = ImageColorGenerator(mask)
    
    wordcloud = WordCloud(max_font_size=130, collocations = False, width = 600, height = 600,
                background_color ='white',
                stopwords = stopwords,
                #mask = transformed_mask,
                min_font_size = 10).generate(wc_str[big_coms[t]])
    
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
plt.show()